<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Play-By-Play-Data" data-toc-modified-id="Load-Play-By-Play-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Play-By-Play Data</a></span></li><li><span><a href="#Extract-+/--Information-for-Lineup-Matchups-and-Apply-Cuts" data-toc-modified-id="Extract-+/--Information-for-Lineup-Matchups-and-Apply-Cuts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extract +/- Information for Lineup Matchups and Apply Cuts</a></span></li><li><span><a href="#Dump-Matchup-Data-to-Files" data-toc-modified-id="Dump-Matchup-Data-to-Files-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dump Matchup Data to Files</a></span></li></ul></div>

In [1]:
import numpy as np
import time
import sys
import os
import pandas as pd
from functools import reduce
from operator import itemgetter
import itertools
import re
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import cm
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [10]:
# Use the player lineups recorded for each possession in 
# play-by-play data to calculate the cumulative plus-minus
# for each lineup matchup "stint". A stint is defined as an 
# offensive or defensive sequence of possessions.
def ExtractPlusMinus(df):
    ht_pm = []
    vt_pm = []
    ht_poss = []
    vt_poss = []
    pts_scored = []
    for num,game in df.groupby(['game', 'year'], sort=False, as_index=False):
        ht_plus_minus = 0
        vt_plus_minus = 0
        ht_stint_poss = 0
        vt_stint_poss = 0
        ht_last_margin = 0
        prev_hl = []
        prev_vl = []
        for ht,vt,hmarg in zip(game.ht_lineup,game.vt_lineup,game.ht_margin):
            hta = sorted(ht.split(','))
            vta = sorted(vt.split(','))
            # If either home or away lineup has changed
            if hta != prev_hl or vta != prev_vl:
                #print('New home lineup:', hta)
                #print('New visiting lineup:', vta)
                ht_plus_minus = 0
                vt_plus_minus = 0
                ht_stint_poss = 0
                vt_stint_poss = 0
            
            pts_scored.append(abs(hmarg-ht_last_margin))
            ht_plus_minus += (hmarg-ht_last_margin)
            vt_plus_minus += -(hmarg-ht_last_margin)
            #print(ht_last_margin, hmarg, ht_plus_minus, pts_scored)
            ht_pm.append(ht_plus_minus)
            vt_pm.append(vt_plus_minus)

            ht_stint_poss += 1
            vt_stint_poss += 1
            ht_poss.append(ht_stint_poss)
            vt_poss.append(vt_stint_poss)
            
            prev_hl = hta
            prev_vl = vta
            ht_last_margin = hmarg
        
    df['ht_stint_pm'] = ht_pm
    df['vt_stint_pm'] = vt_pm
    df['ht_stint_poss'] = ht_poss
    df['vt_stint_poss'] = vt_poss
    df['points_scored'] = pts_scored
    return df

# Grab the data corresponding to the first possession of each lineup matchup stint
def GetMatchupStarts(df):
    dfhead = df.groupby((df[['game', 'year', 'ht_lineup', 'vt_lineup']] != df[['game','year','ht_lineup','vt_lineup']].shift(1)).any(axis=1).cumsum()).head(1).reset_index(drop=True)
    return dfhead

# Grab the data corresponding to the last possession of each lineup matchup stint
# (includes the final plus-minus of the lineup matchup)
def GetMatchupEnds(df):
    dftail = df.groupby((df[['game', 'year', 'ht_lineup', 'vt_lineup']] != df[['game','year','ht_lineup','vt_lineup']].shift(1)).any(axis=1).cumsum()).tail(1).reset_index(drop=True)
    return dftail

# Grab the data corresponding to final row for EACH off/def possession
def GetPossessionEnds(df):
    dftail = df.groupby((df[['game', 'year', 'ht_poss']] != df[['game','year','ht_poss']].shift(1)).any(axis=1).cumsum()).tail(1).reset_index(drop=True)
    return dftail

# DEPRECATED -- extremely slow method
#def PlayerPlusMinus(df, player):
#    df_ht = df[df.ht_lineup.str.contains(player)].time_sec.values
#    df_hpm = df[df.ht_lineup.str.contains(player)].ht_stint_pm.values
#    df_vt = df[df.vt_lineup.str.contains(player)].time_sec.values
#    df_vpm = df[df.vt_lineup.str.contains(player)].vt_stint_pm.values
#    return df_ht, df_hpm, df_vt, df_vpm


# Load Play-By-Play Data

In [3]:
# Load and concatenate all play-by-play data over the last 3 full seasons
df1 = pd.read_csv('../NBA_PBP_Data_2016_2017.csv', index_col=0)
df2 = pd.read_csv('../NBA_PBP_Data_2017_2018.csv', index_col=0)
df3 = pd.read_csv('../NBA_PBP_Data_2018_2019.csv', index_col=0)

df = pd.concat([df1, df2, df3], ignore_index=True)

df.fillna('', inplace=True)

print(df)


         year  game home_team vis_team  Q  time_sec  ht_score  vt_score  \
0        2017     1       CLE      NYK  1       0.0         0         0   
1        2017     1       CLE      NYK  1      20.0         0         2   
2        2017     1       CLE      NYK  1      34.0         0         2   
3        2017     1       CLE      NYK  1      37.0         0         2   
4        2017     1       CLE      NYK  1      44.0         0         2   
...       ...   ...       ...      ... ..       ...       ...       ...   
1675613  2019  1230       POR      SAC  4    2859.0       136       131   
1675614  2019  1230       POR      SAC  4    2859.0       136       131   
1675615  2019  1230       POR      SAC  4    2866.0       136       131   
1675616  2019  1230       POR      SAC  4    2869.0       136       131   
1675617  2019  1230       POR      SAC  4    2880.0       136       131   

         ht_margin  vt_margin  ... ht_flagrants vt_flagrants  ht_2PTA  \
0                0        

# Extract +/- Information for Lineup Matchups and Apply Cuts

In [6]:
# Comb over the PBP dataframe and calculate +/- data 
# for each new lineup stint, for each game
df_new = ExtractPlusMinus(df)
print(df_new)


         year  game home_team vis_team  Q  time_sec  ht_score  vt_score  \
0        2017     1       CLE      NYK  1       0.0         0         0   
1        2017     1       CLE      NYK  1      20.0         0         2   
2        2017     1       CLE      NYK  1      34.0         0         2   
3        2017     1       CLE      NYK  1      37.0         0         2   
4        2017     1       CLE      NYK  1      44.0         0         2   
...       ...   ...       ...      ... ..       ...       ...       ...   
1675613  2019  1230       POR      SAC  4    2859.0       136       131   
1675614  2019  1230       POR      SAC  4    2859.0       136       131   
1675615  2019  1230       POR      SAC  4    2866.0       136       131   
1675616  2019  1230       POR      SAC  4    2869.0       136       131   
1675617  2019  1230       POR      SAC  4    2880.0       136       131   

         ht_margin  vt_margin  ... vt_3PTA ht_2PTM  vt_2PTM  ht_3PTM  vt_3PTM  \
0                0

In [7]:
# Cut out PBP rows where there are not 5 players in both the home and away lineup
df_new = df_new[df_new.ht_lineup.str.split(',').str.len() == 5]
df_new = df_new[df_new.vt_lineup.str.split(',').str.len() == 5]

df_new.to_csv('../NBA_PBP_Data_PlusMinus.csv')
print(df_new)


         year  game home_team vis_team  Q  time_sec  ht_score  vt_score  \
0        2017     1       CLE      NYK  1       0.0         0         0   
1        2017     1       CLE      NYK  1      20.0         0         2   
2        2017     1       CLE      NYK  1      34.0         0         2   
3        2017     1       CLE      NYK  1      37.0         0         2   
4        2017     1       CLE      NYK  1      44.0         0         2   
...       ...   ...       ...      ... ..       ...       ...       ...   
1675613  2019  1230       POR      SAC  4    2859.0       136       131   
1675614  2019  1230       POR      SAC  4    2859.0       136       131   
1675615  2019  1230       POR      SAC  4    2866.0       136       131   
1675616  2019  1230       POR      SAC  4    2869.0       136       131   
1675617  2019  1230       POR      SAC  4    2880.0       136       131   

         ht_margin  vt_margin  ... vt_3PTA ht_2PTM  vt_2PTM  ht_3PTM  vt_3PTM  \
0                0

In [15]:
# Break play-by-play data into units of possessions, then
# store in a file to be loaded for modeling
df_poss = GetPossessionEnds(df_new)
df_poss.to_csv('../NBA_PBP_Data_Possessions.csv')
df_poss[['year', 'game', 'home_team', 'vis_team', 'time_sec', 'ht_score', 'vt_score', 'points_scored', 'ht_play', 'vt_play', 'ht_poss']]


,year,game,home_team,vis_team,time_sec,ht_score,vt_score,points_scored,ht_play,vt_play,ht_poss
0,2017,1,CLE,NYK,20.0,0,2,2,,Rose 1' Driving Layup (2 PTS) (Noah 1 AST),DEF
1,2017,1,CLE,NYK,37.0,0,2,0,,Noah REBOUND (Off:0 Def:1),OFF
2,2017,1,CLE,NYK,45.0,0,4,2,,Porzingis 2' Tip Layup Shot (2 PTS),DEF
3,2017,1,CLE,NYK,61.0,2,4,2,James 11' Jump Shot (2 PTS) (Irving 1 AST),,OFF
4,2017,1,CLE,NYK,62.0,2,4,0,,Rose Out of Bounds Lost Ball Turnover (P1.T1),DEF
...,...,...,...,...,...,...,...,...,...,...,...
720766,2019,1230,POR,SAC,2832.0,135,131,2,Simons 1' Tip Layup Shot (36 PTS),,OFF
720767,2019,1230,POR,SAC,2846.0,135,131,0,Layman REBOUND (Off:0 Def:4),,DEF
720768,2019,1230,POR,SAC,2859.0,136,131,0,,Swanigan REBOUND (Off:3 Def:4),OFF
720769,2019,1230,POR,SAC,2869.0,136,131,0,Labissiere REBOUND (Off:4 Def:11),,DEF


In [6]:
# For a single game, this gives the start and ends of offensive possessions for the home team
#htm_off_starts = np.unique(df[df['ht_poss'] == 'OFF'].time_sec.values.astype(float) - df[df['ht_poss'] == 'OFF'].ht_time_off.values.astype(float))
#vtm_off_starts = np.unique(df[df['vt_poss'] == 'OFF'].time_sec.values.astype(float) - df[df['vt_poss'] == 'OFF'].vt_time_off.values.astype(float))
#print(htm_off_starts, vtm_off_starts)

# PBP data with lineup matchups with too few possessions
# (substitution-only plays, etc.) are removed for each game
min_num_poss = 3
df_newnew = df_new
df_newnew['total_poss'] = df_new.groupby(['game', 'year', 'ht_lineup', 'vt_lineup'], sort=False)['ht_stint_poss'].transform('count')
df_newnew = df_newnew[df_newnew['ht_stint_poss'] > min_num_poss]
df = df_newnew
#print(df)

# Get rows corresponding to start of new lineup matchup (+/- = 0)
dfs = GetMatchupStarts(df)
# Get rows corresponding to end of lineup matchup (final +/-)
dfe = GetMatchupEnds(df)
# Ensure each stint meets minimum possession requirement
dfs = dfs[dfe.ht_stint_poss > min_num_poss]
dfe = dfe[dfe.ht_stint_poss > min_num_poss]
print(dfs, dfe)

# Time average of all play-by-play data (over all games and lineups)
dfm = df.groupby(['time_sec'], as_index=False).mean()
#print(dfm)

# Time average of play-by-play data for each home,away lineup matchup
dfta = df.groupby(['time_sec', 'game', 'year', 'ht_lineup', 'vt_lineup'], as_index=False).mean()
#print(dfta)

# Play-by-play data grouped averaged by quarter
df_by_quarter = df.groupby(['Q'], as_index=False).mean()
#print(df_by_quarter)

# PBP data grouped by home and away teams for team-specific time analysis
dfht = df.groupby(['home_team', 'time_sec'], as_index=False).mean()
dfvt = df.groupby(['vis_team', 'time_sec'], as_index=False).mean()
#print(dfht, dfht.ht_margin)
#print(dfvt, dfvt.vt_margin)
#print(dfht.ht_margin-dfvt.vt_margin)
#print(dfht, dfvt)
#for name in np.unique(dfht.home_team.values): 
#    print(name)

########## DEPRECATED
# Find the PBP rows corresponding to the last possession of each
# lineup matchup, for each game (yielding the cumulative +/- for that lineup)
##dfma = df_new.groupby(['game', 'ht_lineup', 'vt_lineup'], as_index=False).tail(1)
#idx = df_newnew.groupby(['game', 'ht_lineup', 'vt_lineup'], sort=False)['ht_stint_poss'].transform('max') == df_newnew['ht_stint_poss']
#dfma = df_newnew[idx]
#print(dfma)
#print(dfma.ht_lineup)
#print(dfma.vt_lineup)
#print(dfma.ht_stint_pm)

# Mean, first row, and last row of lineups for each game
#dfl = df_newnew.groupby(['ht_lineup', 'vt_lineup'], as_index=False).mean()
#dfhl = df_newnew.groupby(['ht_lineup'], as_index=False).mean()
#dfvl = df_newnew.groupby(['vt_lineup'], as_index=False).mean()
#dfhs = df_newnew.groupby(['game', 'ht_lineup', 'vt_lineup'], as_index=False).head(1)
#dfhs = dfhs.groupby(['ht_lineup'], as_index=False).mean()
#dfvs = df_newnew.groupby(['game', 'ht_lineup', 'vt_lineup'], as_index=False).head(1)
#dfvs = dfvs.groupby(['vt_lineup'], as_index=False).mean()
#dfhe = df_newnew.groupby(['game', 'ht_lineup', 'vt_lineup'], as_index=False).tail(1)
#dfhe = dfhe.groupby(['ht_lineup'], as_index=False).mean()
#dfve = df_newnew.groupby(['game', 'ht_lineup', 'vt_lineup'], as_index=False).tail(1)
#dfve = dfve.groupby(['vt_lineup'], as_index=False).mean()
##########


       year  game home_team vis_team  Q  time_sec  ht_score  vt_score  \
0      2017     1       CLE      NYK  1      37.0         0         2   
1      2017     1       CLE      NYK  1     415.0        14        12   
2      2017     1       CLE      NYK  1     491.0        17        14   
3      2017     1       CLE      NYK  1     557.0        19        16   
4      2017     1       CLE      NYK  1     632.0        23        16   
...     ...   ...       ...      ... ..       ...       ...       ...   
98072  2019  1230       POR      SAC  4    2256.0       104       115   
98073  2019  1230       POR      SAC  4    2400.0       113       117   
98074  2019  1230       POR      SAC  4    2429.0       116       117   
98075  2019  1230       POR      SAC  4    2565.0       122       117   
98076  2019  1230       POR      SAC  4    2624.0       124       119   

       ht_margin  vt_margin  ... vt_3PTA ht_2PTM  vt_2PTM  ht_3PTM  vt_3PTM  \
0             -2          2  ...       0    

# Dump Matchup Data to Files

In [7]:
# Write lineup matchup +/- data to files
dfpm = dfe.copy()
dfpm['ht_pm_ph'] = dfpm.ht_stint_pm.values*100./dfpm.ht_stint_poss.values
dfpm['vt_pm_ph'] = dfpm.vt_stint_pm.values*100./dfpm.ht_stint_poss.values
#dfpm = dfpm.groupby(['ht_lineup', 'vt_lineup'], as_index=False).mean()
dfpm.to_csv('../NBA_Full_Matchup_Data.csv')

# More simplified dataframe
dfpm = dfe[['year', 'game', 'ht_lineup', 'vt_lineup', 'ht_stint_pm', 'vt_stint_pm', 'ht_stint_poss']]
dfpm['ht_pm_ph'] = dfpm.ht_stint_pm.values*100./dfpm.ht_stint_poss.values
dfpm['vt_pm_ph'] = dfpm.vt_stint_pm.values*100./dfpm.ht_stint_poss.values
#dfpm = dfpm.groupby(['ht_lineup', 'vt_lineup'], as_index=False).mean()
dfpm.to_csv('../NBA_Matchup_PlusMinus.csv')
print(dfpm)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


       year  game                                          ht_lineup  \
0      2017     1  JR Smith,Kevin Love,Kyrie Irving,LeBron James,...   
1      2017     1  JR Smith,Kyrie Irving,LeBron James,Richard Jef...   
2      2017     1  Iman Shumpert,Kyrie Irving,LeBron James,Richar...   
3      2017     1  Iman Shumpert,Kevin Love,Mike Dunleavy,Richard...   
4      2017     1  Iman Shumpert,Kevin Love,Mike Dunleavy,Richard...   
...     ...   ...                                                ...   
98072  2019  1230  Jake Layman,Meyers Leonard,Gary Trent Jr.,Anfe...   
98073  2019  1230  Anfernee Simons,Gary Trent Jr.,Jake Layman,Mey...   
98074  2019  1230  Anfernee Simons,Gary Trent Jr.,Jake Layman,Mey...   
98075  2019  1230  Anfernee Simons,Gary Trent Jr.,Jake Layman,Mey...   
98076  2019  1230  Anfernee Simons,Gary Trent Jr.,Jake Layman,Mey...   

                                               vt_lineup  ht_stint_pm  \
0      Carmelo Anthony,Courtney Lee,Derrick Rose,Joak...      